In [1]:
import dask
from dask import dataframe as dd
import warnings

from dask.distributed import LocalCluster

warnings.filterwarnings(action='ignore', category=FutureWarning)

# with dask.config.set({"dataframe.shuffle.method": "tasks"}):
dask.config.set({
    "distributed.workers.memory.target": 0.75,
    "distributed.workers.memory.spill": 0.85,
    "distributed.workers.memory.terminate": 0.98,
    "dataframe.shuffle.method": "p2p"})
client = LocalCluster(n_workers=4).get_client() # reduce # of worker for larger worker memory

2025-01-10 13:06:04,852 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 245ebc6b4cc13edf382dd4fd6ea97288 initialized by task ('shuffle-transfer-245ebc6b4cc13edf382dd4fd6ea97288', 7) executed on worker tcp://127.0.0.1:54480
2025-01-10 13:06:07,028 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 245ebc6b4cc13edf382dd4fd6ea97288 deactivated due to stimulus 'task-finished-1736489167.028161'


In [2]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 12,Total memory: 29.66 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:54459,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 12
Started: Just now,Total memory: 29.66 GiB
Comm: tcp://127.0.0.1:54479,Total threads: 3
Dashboard: http://127.0.0.1:54484/status,Memory: 7.42 GiB
Nanny: tcp://127.0.0.1:54462,


In [3]:
%run "C:\Users\o_thanakrit.b\OneDrive - KASIKORNBANKGROUP\เอกสาร\code_and_finding\template.ipynb"

In [4]:
risk_full = dd.read_parquet(mart_risk_path/"risk_new_full2_blc_profit_usage.parquet")
card_type = dd.read_parquet(staging_path/"card_type.parquet")
ecl = dd.read_parquet(staging_path/"ecl_202112_202404.parquet")

# Check data configuration

In [5]:
print(f"Division know : {risk_full.known_divisions}, number of partition : {risk_full.npartitions}")

Division know : False, number of partition : 249


In [6]:
risk_full.partitions[0].memory_usage(deep=True).compute().apply(dask.utils.format_bytes)

Index                  132 B
uniq_key            3.88 MiB
opn_dt              3.88 MiB
card_status         6.61 MiB
past_due_amt        3.88 MiB
stmt_amt            3.88 MiB
mth_end_bal         3.88 MiB
crn_pymt_due_amt    3.88 MiB
util                3.88 MiB
cr_lmt_amt          3.88 MiB
acq_cnl             6.42 MiB
lmt_at_app          3.88 MiB
mnth_cd             1.94 MiB
MOB                 3.88 MiB
behv                7.92 MiB
pymt_amt            3.88 MiB
dlq_bck             6.90 MiB
ews                 4.49 MiB
b_scor              4.28 MiB
blc_cd              4.69 MiB
usage               3.88 MiB
profit              3.88 MiB
dtype: object

In [7]:
dask.utils.format_bytes(risk_full.partitions[0].memory_usage(deep=True).compute().sum())

'93.65 MiB'

# Test distributed computation

## Univariate stats

In [8]:
risk_full.groupby(["MOB"])["profit"].agg(["min", "max", "mean", "std", "count", "size"]).compute()

,min,max,mean,std,count,size
MOB,,,,,,
112.0,-894033.259823,282638.960000,158.739698,2167.645656,496703,496703
75.0,-343778.907447,209982.208449,175.126430,1798.006515,334230,334230
52.0,-150394.521088,585629.429237,80.270897,1377.228642,1036520,1036520
35.0,-551346.552140,552635.185751,93.503238,1553.366551,976902,976902
7.0,-147556.056876,139345.865751,69.104988,1030.837296,1228923,1228923
...,...,...,...,...,...,...
722.0,-69.885029,-69.885029,-69.885029,NaN,1,1
723.0,-89.269725,-89.269725,-89.269725,NaN,1,1
726.0,-102.591827,-102.591827,-102.591827,NaN,1,1


## Mode take longti

In [ ]:
risk_full[["profit"]].mode().compute()

In [8]:
card_type.dtypes

uniq_key                     int64
card_tp            string[pyarrow]
issur_card_lvl2    string[pyarrow]
dtype: object

In [11]:
card_type.card_tp.unique().compute()

0                         PTT
1                    PLATINUM
2                    One Siam
3                     CLASSIC
4                        BDMS
5                  KING POWER
6                     Passion
7                      Wisdom
8                        Line
9                   SIGNATURE
10                     Shopee
11                        SME
12                       GOLD
13    MUANG THAI SMILE CREDIT
14                       Corp
15                        CGA
16                      Amway
17                  THAMMASAT
18                   Infinite
Name: card_tp, dtype: string

In [49]:
card_type_pdf = card_type.compute()

In [51]:
(card_type_pdf
 .assign(tag_card_tp = lambda x : np.where(x["card_tp"].isin(["CLASSIC", "PLATINUM", "GOLD", "SIGNATURE", "Infinite"]), "Generic", x["card_tp"]))
).head()

,uniq_key,card_tp,issur_card_lvl2,tag_card_tp
0,1,PTT,PLATINUM,PTT
1,2,PTT,PLATINUM,PTT
2,3,PLATINUM,PLATINUM,Generic
3,4,PLATINUM,PLATINUM,Generic
4,5,One Siam,SIGNATURE,One Siam


In [72]:
card_type["tag_card_tp"] = card_type["card_tp"].isin(["CLASSIC", "PLATINUM", "GOLD", "SIGNATURE", "Infinite"])

In [77]:
card_type.assign(tag_card_tp = lambda x : x["card_tp"].mask( x["card_tp"].isin(["CLASSIC", "PLATINUM", "GOLD", "SIGNATURE", "Infinite"]), "Generic")).head()

,uniq_key,card_tp,issur_card_lvl2,tag_card_tp
0,1,PTT,PLATINUM,PTT
1,2,PTT,PLATINUM,PTT
2,3,PLATINUM,PLATINUM,Generic
3,4,PLATINUM,PLATINUM,Generic
4,5,One Siam,SIGNATURE,One Siam


In [ ]:
card_type.assign(tag_card_tp = lambda x : x["card_tp"].where( x["card_tp"].isin(["CLASSIC", "PLATINUM", "GOLD", "SIGNATURE", "Infinite"]), "Generic")).head()

In [10]:
ecl.dtypes

uniq_key        int64
ECL_OVRD_W    float64
mnth_cd         int32
dtype: object

In [11]:
apr24_ecl = ecl.query("mnth_cd == 202404").loc[:, ["uniq_key", "ECL_OVRD_W"]]

In [12]:
apr24_ecl["ECL_OVRD_W"].sum().compute()

8818415222.777327

In [13]:
apr24_ecl["uniq_key"].count().compute()

4114628

In [14]:
apr24_status_blck_pdf = apr24_status_blck.compute()
apr24_ecl_pdf = apr24_ecl.compute()

In [15]:
status_blck_ecl = apr24_status_blck_pdf.merge(apr24_ecl_pdf, on="uniq_key", how="left")

In [17]:
status_blck_ecl.dtypes

uniq_key                 int64
card_status    string[pyarrow]
blc_cd         string[pyarrow]
ECL_OVRD_W             float64
dtype: object

In [19]:
status_blck_ecl = status_blck_ecl.fillna({"ECL_OVRD_W":0.0})

In [21]:
status_blck_ecl.groupby(["card_status", "blc_cd"], as_index=False).agg(n_card = ("uniq_key", "count"), tt_ecl = ("ECL_OVRD_W", "sum"))

,card_status,blc_cd,n_card,tt_ecl
0,Invalid,B,933,1.206941e+07
1,Invalid,C,23,3.492986e+06
2,Invalid,D,7370,1.800725e+07
3,Invalid,F,809,1.599575e+05
4,Invalid,H,16,3.457216e+05
5,Invalid,I,1,5.643167e+02
6,Invalid,J,33352,8.456106e+05
7,Invalid,L,412911,2.656537e+07
8,Invalid,M,6865,2.968408e+07
9,Invalid,N,187,6.569545e+04


In [22]:
status_blck_ecl.groupby(["card_status", "blc_cd"], as_index=False).agg(n_card = ("uniq_key", "count"), tt_ecl = ("ECL_OVRD_W", "sum")).to_csv("apr24_card_status_blck_ecl.csv")

In [27]:
apr24_status_blck = risk_full.query("mnth_cd == 202404").loc[:, ["uniq_key", "card_status", "blc_cd"]]
apr24_ecl = ecl.query("mnth_cd == 202404").loc[:, ["uniq_key", "ECL_OVRD_W"]]
jan24_blck = risk_full.query("mnth_cd == 202401").loc[:, ["uniq_key", "card_status", "blc_cd"]].rename(columns = {"card_status":"card_status_jan24", "blc_cd":"blc_cd_jan24"})
oct23_blck = risk_full.query("mnth_cd == 202312").loc[:, ["uniq_key", "card_status", "blc_cd"]].rename(columns = {"card_status":"card_status_oct23", "blc_cd":"blc_cd_oct23"})

In [28]:
apr24_status_blck_pdf = apr24_status_blck.compute()
apr24_ecl_pdf = apr24_ecl.compute()
jan24_blck_pdf = jan24_blck.compute()
oct23_blck_pdf = oct23_blck.compute()

In [47]:
apr24_status_blck_ecl_back_date = \
(apr24_status_blck_pdf
 .merge(apr24_ecl_pdf, on="uniq_key", how="left")
 .fillna({"ECL_OVRD_W":0.0})
 .merge(jan24_blck_pdf, on="uniq_key", how="left")
 .merge(oct23_blck_pdf, on="uniq_key", how="left")
 .fillna({"blc_cd_jan24":"NA", "blc_cd_oct23":"NA"})
)

In [48]:
apr24_status_blck_ecl_back_date.dtypes

uniq_key                       int64
card_status          string[pyarrow]
blc_cd               string[pyarrow]
ECL_OVRD_W                   float64
card_status_jan24    string[pyarrow]
blc_cd_jan24         string[pyarrow]
card_status_oct23    string[pyarrow]
blc_cd_oct23         string[pyarrow]
dtype: object

In [49]:
(apr24_status_blck_ecl_back_date
 .groupby(["card_status", "blc_cd", "blc_cd_jan24", "blc_cd_oct23"], as_index=False)
 .agg(n_card = ("uniq_key", "count"),
      tt_ecl = ("ECL_OVRD_W", "sum")
      )
).to_csv("blc_duration_apr24_jan24_oct23.csv")

In [39]:
apr24_status_blck_ecl_back_date["blc_cd"].count()

4390810

In [38]:
(apr24_status_blck_pdf
 .assign(b_scor_flg = lambda x : np.where(x["b_scor"].isna(), "N", "Y"))
 .merge(apr24_ecl_pdf, on="uniq_key", how="left")
 .fillna({"ECL_OVRD_W":0.0, "dlq_bck":"<NA>"})
 .groupby(["card_status", "blc_cd", "dlq_bck", "b_scor_flg"], as_index=False)
 .agg(n_card = ("uniq_key", "count"), tt_ecl = ("ECL_OVRD_W", "sum"))
).to_csv("apr24_blc_cd_dpd_b_scor.csv", index=False)